# Retrieve 5j7v

In [1]:

import sys
import pickle
import numpy as np
import gc
import glob
import scipy.stats
import sklearn.cluster
import shutil
sys.path.append('..')
sys.path.append('../../')
sys.path.append('../../InchingLiteInteger/')
import InchingLiteInteger.Burn.Orthogonalization.T3
import InchingLiteInteger.Burn.Orthogonalization.T2
import InchingLiteInteger.Burn.Orthogonalization.T1
import InchingLiteInteger.Burn.Krylov.T3
import InchingLiteInteger.util
from InchingLiteInteger.Fuel.Coordinate.T1 import HeigvecOne_BoxCoxMagnitude


import tqdm

import openmm.app as mmapp
import mdtraj


/home/homingla/anaconda3/envs/V0/envs/Inching23/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Doesn ot complete

In [2]:

import time
def SaveReducedModel(inputpdbfn = "../../DataRepo/CifByAtomCount/5iv7.cif",
                     outputfolder = "./",
                     subsetkeywords = ["name CA", ]):
    
    st = time.time()
    print("reading postino")
    pdb = mmapp.pdbxfile.PDBxFile(inputpdbfn)
    fileunit = 1.0  

    pdbid = inputpdbfn.split("/")[-1].split(".")[0]

    pdb_position = pdb.getPositions(asNumpy=True, frame=0) * fileunit
    print("finish postino", time.time() - st)
    st = time.time()
    print("Reading inital topology")
    # NOTE This is a redundant step we need cleaning but not at openmm
    top = mdtraj.Topology.from_openmm(pdb.topology) # TODO Copy the whole class from mdtraj to a util to reduce the load and reliance
    topology_df, bond_df = top.to_dataframe()
    topology_df.loc[:, ['x', 'y', 'z']] = pdb_position
    print("Finish inital topology", time.time() - st)
    position = topology_df[['x', 'y', 'z']].values
    print(bond_df.shape, topology_df.shape)
    #print(bond_df)
    #return

    #return
    for i_subset in range(len(subsetkeywords)):
        st = time.time()
        print("Compiling position")

        #print(bond_df[idx_sele,:], )


        idx_sele = top.select(subsetkeywords[i_subset])
        position_sele = position[idx_sele,:][None,:,:]

        from openmm import Vec3
        from openmm.unit import nanometers
        # Overwrite exisiting
        new_pos = []

        for i in tqdm.tqdm(range(position_sele.shape[1])):
                #print(position_sele[t,i,0])
                new_pos.append(
                    Vec3(
                        float(position_sele[0,i,0]), 
                        float(position_sele[0,i,1]), 
                        float(position_sele[0,i,2]))*fileunit)


        print("Finishe position", time.time() - st)


        st = time.time()
        print("Compiling subset topology")
        """
        top_sele = top.subset(idx_sele)
        #print(topology_df.iloc[idx_sele])
        """
        topology_df_sele = topology_df.iloc[idx_sele].reset_index()
        print("Finishe subset topology dataframe mdtrah", time.time() - st)
        top_sele = mdtraj.Topology.from_dataframe(topology_df_sele, None)# bond_df[idx_sele,:])
        
        print("Finishe subset topology mdtraj", time.time() - st)
        openmmtop_sele = top_sele.to_openmm()
        print("Finishe subset topology", time.time() - st)




        # NOTE Remake positoopn

        print("WRITTING")
        #continue
        with open("%s/%s_%s.cif" %(outputfolder, pdbid, subsetkeywords[i_subset].replace(" ", "")), 'w') as tempfile:
            mmapp.pdbxfile.PDBxFile.writeHeader(openmmtop_sele, file=tempfile,)
            mmapp.pdbxfile.PDBxFile.writeModel(
                            openmmtop_sele, new_pos, 
                            file=tempfile, keepIds=True, modelIndex = i)
        print("Finishe writing", time.time() - st)

SaveReducedModel(inputpdbfn = "../../DataRepo/CifShowcase/5j7v/5j7v-assembly1.cif",
                 outputfolder = "../../DataRepo/CifShowcase/5j7v/",
                     subsetkeywords = ["name CA", "backbone"])


reading postino
